<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Calculate distance and price
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Sheets+-+Calculate+distance+and+price:+Error+short+description">Bug report</a>

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet #dataframe #google_maps_api #routes_api

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Last update:** 2023-07-27 (Created: 2023-07-27)

**Description:** This template determines the cost and distance between location extracted from a Google Sheet. It uses the Routes API to estimate the price depending on the distance between sites and outputs the updated Google Sheet with distances and prices.

## Input

### Import libraries

In [1]:
import naas
from naas_drivers import gsheet
import requests

### Setup variables
**Get your Google Maps API key**

1. Follow the steps in the link - https://developers.google.com/maps/documentation/routes/cloud-setup
2. Sign up for an account with the Routes API provider.
3. Make sure to enable Routes API as it is in the "Additional APIs" section in the google cloud console.
4. Check the available countries https://developers.google.com/maps/documentation/routes/calculate_toll_fees#toll-locations

- `api_key`: This variable holds the Google Cloud Platform (GCP) API key. The key is retrieved from the secret variable using the Naas secret manager.

**Setup Google Sheets**

Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

- `spreadsheet_url`: Google Sheets spreadsheet URL
- `sheet_name`: Google Sheets sheet name
- `origin_column`: Origin column name
- `destination_column`: Destination column name

In [2]:
api_key = naas.secret.get("GCP_MAP_API_KEY")  # Read API key from the secret variable
spreadsheet_url = "https://docs.google.com/spreadsheets/x/xxxxxxxxxxxxxx/edit?usp=sharing"
sheet_name = "Random"
origin_column = "Origin"
destination_column = "Destination"

## Model

### Get data from Google Sheets spreadsheet

In [ ]:
df = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Row fetched:", len(df))
df.head(1)

### Convert addresses to coordinates

In [4]:
def get_coordinates(address):
    geocoding_api_endpoint = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": api_key}
    response = requests.get(geocoding_api_endpoint, params=params)
    data = response.json()
    if data["status"] == "OK":
        result = data["results"][0]
        lat = result["geometry"]["location"]["lat"]
        lng = result["geometry"]["location"]["lng"]
        return lat, lng
    else:
        return None

### Update the Google Sheet with the relevant coordinates

In [5]:
for index, row in df.iterrows():
    origin_address = row[origin_column]
    destination_address = row[destination_column]
    
    origin_coordinates = get_coordinates(origin_address)
    if origin_coordinates:
        df.at[index, "Origin lat"] = origin_coordinates[0]
        df.at[index, "Origin lng"] = origin_coordinates[1]
    
    destination_coordinates = get_coordinates(destination_address)
    if destination_coordinates:
        df.at[index, "Destination lat"] = destination_coordinates[0]
        df.at[index, "Destination lng"] = destination_coordinates[1]

### Retrieve distance and toll information from Routes API and update the sheet

In [6]:
api_endpoint = 'https://routes.googleapis.com/directions/v2:computeRoutes'

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': api_key,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.travelAdvisory.tollInfo,routes.legs.travelAdvisory.tollInfo'
}

for index, row in df.iterrows():
    origin_lat = row["Origin lat"]
    origin_lng = row["Origin lng"]
    destination_lat = row["Destination lat"]
    destination_lng = row["Destination lng"]

    payload = {
        "origin": {
            "location": {
                "latLng": {
                    "latitude": origin_lat,
                    "longitude": origin_lng
                }
            }
        },
        "destination": {
            "location": {
                "latLng": {
                    "latitude": destination_lat,
                    "longitude": destination_lng
                }
            }
        },
        "travelMode": "DRIVE",
        "extraComputations": ["TOLLS"],
        "routeModifiers": {
            "vehicleInfo": {
                "emissionType": "GASOLINE"
            },
            "tollPasses": [
               # Add relevant toll passes if applicable
            ]
        }
    }

    response = requests.post(api_endpoint, json=payload, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            route = data["routes"][0]
            distance_meters = route.get("distanceMeters")
            if distance_meters:
                distance_km = distance_meters / 1000
                df.loc[index, "Distance"] = f"{distance_km:.2f} km"
            if 'travelAdvisory' in route:
                toll_info = route.get("travelAdvisory", {}).get("tollInfo", {})
                toll_info = route.get("travelAdvisory", {}).get("tollInfo", {})
                estimated_price = toll_info.get("estimatedPrice", [])
                if estimated_price:
                    currency_code = estimated_price[0].get("currencyCode", "")
                    toll_cost = float(estimated_price[0].get("units", 0))
                    formatted_toll_cost = '{:.2f}'.format(toll_cost)
                    df.loc[index, "Toll costs"] = currency_code + ' ' + formatted_toll_cost
            else:
                df.loc[index, "Toll costs"] = "Not available"
        else:
            print(f"No routes found in the response for row {index+2}")
    else:
        print(f"Route calculation request failed for row {index+2} with status code: {response.status_code}")

## Output

### Update Google Sheet spreadsheet

In [ ]:
result = gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=df, append=False)
if "error" in result:
    print("Failed to save the sheet.")
else:
    print("Saved successfully.")